In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from keras.callbacks import ModelCheckpoint

In [2]:
Sequential = tf.keras.models.Sequential
optimizers = tf.keras.optimizers
metrics = tf.keras.metrics
Dense = tf.keras.layers.Dense
Flatten = tf.keras.layers.Flatten
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

In [3]:
# CONSTANT
LEARNING_RATE = 0.0001
EPOCHS = 100
MODEL_SAVE = "Resnet50v2_ai_tm_lr_0.0001_augmentation"  # 저장할 h5파일 이름
PATH = 'D:\\Ai_hub\\TEAM_Coprime\\data\\ai_tm\\train' # 경로 수정

np.random.seed(3)

# 데이터 준비


In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                  rotation_range = 30, 
                                  width_shift_range=0.1, 
                                  height_shift_range=0.1,
                                  shear_range=0.2, 
                                  zoom_range=0.3,
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest',
                                  validation_split=0.3
                                  )

train_generator = train_datagen.flow_from_directory(
        PATH,  
        target_size=(256, 256),  
        batch_size=256, 
        class_mode='categorical',
        subset='training')

Found 26624 images belonging to 95 classes.


In [5]:
validation_generator = train_datagen.flow_from_directory(
        PATH,
        target_size=(256, 256),    
        class_mode='categorical',
        batch_size=256,
        subset='validation')

Found 11348 images belonging to 95 classes.


## Resnet50V2

In [6]:
# ResNet50V2 모델 불러오기
transfer_model = tf.keras.applications.ResNet50V2(weights='imagenet', include_top = False, input_shape = (256,256,3))
transfer_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 130, 130, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [7]:
transfer_model.trainable = False

model = Sequential()
model.add(transfer_model)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(95, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 8, 8, 2048)        23564800  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                8388672   
_________________________________________________________________
dense_1 (Dense)              (None, 95)                6175      
Total params: 31,959,647
Trainable params: 8,394,847
Non-trainable params: 23,564,800
_________________________________________________________________


In [8]:
model.compile(loss='categorical_crossentropy',
                       optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
                       metrics=['accuracy'])

In [9]:
filename = 'check_point/checkpoint-epoch-{}-trial-001.h5'.format(EPOCHS)
checkpoint = ModelCheckpoint(filename,             # file명을 지정합니다
                             monitor='val_accuracy',   # val_accuracy 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=True,  # 가장 best 값만 저장합니다
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )

In [ ]:
history = model.fit_generator(train_generator, epochs=EPOCHS, validation_data=validation_generator, callbacks=[checkpoint])

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

C:\ProgramData\Anaconda3\envs\ai_hub2\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
104/104 [==============================] - 1312s 13s/step - loss: 3.4499 - accuracy: 0.2232 - val_loss: 2.8592 - val_accuracy: 0.3423

Epoch 00001: val_accuracy improved from -inf to 0.34226, saving model to check_point\checkpoint-epoch-100-trial-001.h5


C:\ProgramData\Anaconda3\envs\ai_hub2\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
104/104 [==============================] - 1307s 13s/step - loss: 1.6129 - accuracy: 0.6196 - val_loss: 1.9577 - val_accuracy: 0.5472

Epoch 00002: val_accuracy improved from 0.34226 to 0.54723, saving model to check_point\checkpoint-epoch-100-trial-001.h5
Epoch 3/100
104/104 [==============================] - 1302s 13s/step - loss: 0.8578 - accuracy: 0.7814 - val_loss: 1.5893 - val_accuracy: 0.6391

Epoch 00003: val_accuracy improved from 0.54723 to 0.63906, saving model to check_point\checkpoint-epoch-100-trial-001.h5
Epoch 4/100
104/104 [==============================] - 1306s 13s/step - loss: 0.5395 - accuracy: 0.8585 - val_loss: 1.4205 - val_accuracy: 0.6898

Epoch 00004: val_accuracy improved from 0.63906 to 0.68981, saving model to check_point\checkpoint-epoch-100-trial-001.h5
Epoch 5/100
104/104 [==============================] - 1304s 13s/step - loss: 0.3895 - accuracy: 0.8964 - val_loss: 1.3752 - val_accuracy: 0.7067

Epoch 00005: val_accuracy improved from 0.6898

In [ ]:
model.save(f'{MODEL_SAVE}.h5')

In [ ]:
history = model.load_model('check_point/checkpoint-epoch-100-trial-001.h5')

In [ ]:
# 시각화
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
y_vloss = history.history['val_loss']
y_loss = history.history['loss']


x_len = np.arange(len(y_loss))
plt.plot(x_len, acc, marker='.', c='red', label='acc')
plt.plot(x_len, val_acc, marker='.', c='yellow', label='val_acc')
plt.plot(x_len, y_loss, marker='.', c='blue', label='loss')
plt.plot(x_len, y_vloss, marker='.', c='turquoise', label='val_loss')

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss/acc')
plt.show()